In [186]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.special as special 
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [187]:
#Cargar archivo csv des seaborn

# Verificar sin indice 
df= pd.read_csv('Boston_libre de atipicos.csv')
df

,Unnamed: 0,last_scraped,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_verifications,...,availability_60,availability_90,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_checkin,review_scores_communication,review_scores_location
0,0,2024-12-20,2008-12-03,"Massachusetts, United States",within a day,100%,22%,t,East Boston,"['email', 'phone']",...,0,17,26.0,1.0,0.0,4.96,5.00,5.00,4.96,4.85
1,1,2024-12-20,2009-02-19,"Boston, MA",within an hour,100%,99%,t,Roxbury,"['email', 'phone']",...,50,80,132.0,10.0,0.0,4.81,4.88,4.95,4.89,4.56
2,2,2024-12-20,2009-02-19,"Boston, MA",within an hour,100%,99%,t,Roxbury,"['email', 'phone']",...,52,82,135.0,8.0,0.0,4.81,4.83,4.91,4.95,4.53
3,3,2024-12-20,2009-07-22,"Boston, MA",within a few hours,100%,49%,t,Beacon Hill,"['email', 'phone']",...,0,0,29.0,1.0,0.0,4.69,4.69,4.93,4.97,4.97
4,4,2024-12-20,2009-09-16,"Boston, MA",within a day,91%,58%,f,Back Bay,"['email', 'phone']",...,59,89,9.0,0.0,0.0,4.80,4.90,4.90,4.90,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4421,4421,2024-12-20,2020-10-28,"Boston, MA",within an hour,100%,97%,f,Harvard Square,"['email', 'phone']",...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4422,4422,2024-12-20,2020-10-28,"Boston, MA",within an hour,100%,97%,f,Harvard Square,"['email', 'phone']",...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4423,4423,2024-12-20,2018-08-29,"Boston, MA",within an hour,100%,98%,f,Theater District,"['email', 'phone']",...,57,87,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4424,4424,2024-12-20,2016-05-29,"Boston, MA",within an hour,100%,100%,t,Roslindale,"['email', 'phone']",...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90


In [188]:
df = pd.read_csv('Boston_libre de atipicos.csv')
df['host_acceptance_rate']=df['host_acceptance_rate'].str.replace('%', '')
df['host_acceptance_rate']= df['host_acceptance_rate'].astype(float)
df['price']=df['price'].str.replace('$', '')
df['price']=df['price'].str.replace(',', '')
df['host_response_rate']=df['host_response_rate'].str.replace('%', '')
df['host_response_rate']= df['host_response_rate'].astype(float)
df['price']= df['price'].astype(float)
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [189]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4426 entries, 0 to 4425
Data columns (total 46 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   last_scraped                 4426 non-null   object 
 1   host_since                   4426 non-null   object 
 2   host_location                4426 non-null   object 
 3   host_response_time           4426 non-null   object 
 4   host_response_rate           4426 non-null   float64
 5   host_acceptance_rate         4426 non-null   float64
 6   host_is_superhost            4426 non-null   object 
 7   host_neighbourhood           4426 non-null   object 
 8   host_verifications           4426 non-null   object 
 9   host_identity_verified       4426 non-null   object 
 10  neighbourhood                4426 non-null   object 
 11  neighbourhood_cleansed       4426 non-null   object 
 12  property_type                4426 non-null   object 
 13  room_type         

In [190]:
df['room_type'].unique()

array(['Entire home/apt', 'Private room', 'Shared room', 'Hotel room'],
      dtype=object)

In [191]:
df['has_availability'].unique()

array(['t', 'f'], dtype=object)

In [192]:
df['host_is_superhost'].unique()

array(['t', 'f'], dtype=object)

In [193]:
from funpymodeling.exploratory import freq_tbl
tableA = freq_tbl (df['room_type'])


#Elimino las columnas percentage y cumulative perc para trabajar con las frecuencias
tableA = tableA.drop(['percentage','cumulative_perc'],axis=1)
tableA

,room_type,frequency
0,Entire home/apt,2999
1,Private room,1360
2,Hotel room,63
3,Shared room,4


In [194]:
from funpymodeling.exploratory import freq_tbl
tableA = freq_tbl (df['has_availability'])


#Elimino las columnas percentage y cumulative perc para trabajar con las frecuencias
tableA = tableA.drop(['percentage','cumulative_perc'],axis=1)
tableA

,has_availability,frequency
0,t,4422
1,f,4


In [195]:
from funpymodeling.exploratory import freq_tbl
tableA = freq_tbl (df['host_is_superhost'])


#Elimino las columnas percentage y cumulative perc para trabajar con las frecuencias
tableA = tableA.drop(['percentage','cumulative_perc'],axis=1)
tableA

,host_is_superhost,frequency
0,f,2985
1,t,1441


In [196]:
from funpymodeling.exploratory import freq_tbl
tableA = freq_tbl (df['host_verifications'])


#Elimino las columnas percentage y cumulative perc para trabajar con las frecuencias
tableA = tableA.drop(['percentage','cumulative_perc'],axis=1)
tableA

,host_verifications,frequency
0,"['email', 'phone']",3192
1,"['email', 'phone', 'work_email']",898
2,['phone'],266
3,"['phone', 'work_email']",67
4,['email'],3


In [197]:
# Limpiamos los strings quitando corchetes y comillas
df['host_verifications'] = df['host_verifications'].str.replace(r"[\[\]']", "", regex=True)

# Y luego reemplazamos las comas por underscores o simplemente quitarlas
df['host_verifications'] = df['host_verifications'].str.replace(", ", "_")

In [198]:
df['host_verifications'].unique()

array(['email_phone', 'email_phone_work_email', 'phone',
       'phone_work_email', 'email'], dtype=object)

In [199]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
ordinal=OrdinalEncoder(categories=[['Entire home/apt', 'Private room', 'Hotel room', 'Shared room']])

#Se transforman los datos 
df['room_type']=ordinal.fit_transform(df[['room_type']])

#Cambiar el tipo de dato a un entero 
df['room_type']=df['room_type'].astype('int64')
df

,last_scraped,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_verifications,host_identity_verified,...,availability_60,availability_90,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_checkin,review_scores_communication,review_scores_location
0,2024-12-20,2008-12-03,"Massachusetts, United States",within a day,100.0,22.0,t,East Boston,email_phone,f,...,0,17,26.0,1.0,0.0,4.96,5.00,5.00,4.96,4.85
1,2024-12-20,2009-02-19,"Boston, MA",within an hour,100.0,99.0,t,Roxbury,email_phone,t,...,50,80,132.0,10.0,0.0,4.81,4.88,4.95,4.89,4.56
2,2024-12-20,2009-02-19,"Boston, MA",within an hour,100.0,99.0,t,Roxbury,email_phone,t,...,52,82,135.0,8.0,0.0,4.81,4.83,4.91,4.95,4.53
3,2024-12-20,2009-07-22,"Boston, MA",within a few hours,100.0,49.0,t,Beacon Hill,email_phone,t,...,0,0,29.0,1.0,0.0,4.69,4.69,4.93,4.97,4.97
4,2024-12-20,2009-09-16,"Boston, MA",within a day,91.0,58.0,f,Back Bay,email_phone,t,...,59,89,9.0,0.0,0.0,4.80,4.90,4.90,4.90,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4421,2024-12-20,2020-10-28,"Boston, MA",within an hour,100.0,97.0,f,Harvard Square,email_phone,t,...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4422,2024-12-20,2020-10-28,"Boston, MA",within an hour,100.0,97.0,f,Harvard Square,email_phone,t,...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4423,2024-12-20,2018-08-29,"Boston, MA",within an hour,100.0,98.0,f,Theater District,email_phone,t,...,57,87,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4424,2024-12-20,2016-05-29,"Boston, MA",within an hour,100.0,100.0,t,Roslindale,email_phone,t,...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90


In [200]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
ordinal=OrdinalEncoder(categories=[['t', 'f']])

#Se transforman los datos 
df['has_availability']=ordinal.fit_transform(df[['has_availability']])

#Cambiar el tipo de dato a un entero 
df['has_availability']=df['has_availability'].astype('int64')
df

,last_scraped,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_verifications,host_identity_verified,...,availability_60,availability_90,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_checkin,review_scores_communication,review_scores_location
0,2024-12-20,2008-12-03,"Massachusetts, United States",within a day,100.0,22.0,t,East Boston,email_phone,f,...,0,17,26.0,1.0,0.0,4.96,5.00,5.00,4.96,4.85
1,2024-12-20,2009-02-19,"Boston, MA",within an hour,100.0,99.0,t,Roxbury,email_phone,t,...,50,80,132.0,10.0,0.0,4.81,4.88,4.95,4.89,4.56
2,2024-12-20,2009-02-19,"Boston, MA",within an hour,100.0,99.0,t,Roxbury,email_phone,t,...,52,82,135.0,8.0,0.0,4.81,4.83,4.91,4.95,4.53
3,2024-12-20,2009-07-22,"Boston, MA",within a few hours,100.0,49.0,t,Beacon Hill,email_phone,t,...,0,0,29.0,1.0,0.0,4.69,4.69,4.93,4.97,4.97
4,2024-12-20,2009-09-16,"Boston, MA",within a day,91.0,58.0,f,Back Bay,email_phone,t,...,59,89,9.0,0.0,0.0,4.80,4.90,4.90,4.90,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4421,2024-12-20,2020-10-28,"Boston, MA",within an hour,100.0,97.0,f,Harvard Square,email_phone,t,...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4422,2024-12-20,2020-10-28,"Boston, MA",within an hour,100.0,97.0,f,Harvard Square,email_phone,t,...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4423,2024-12-20,2018-08-29,"Boston, MA",within an hour,100.0,98.0,f,Theater District,email_phone,t,...,57,87,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4424,2024-12-20,2016-05-29,"Boston, MA",within an hour,100.0,100.0,t,Roslindale,email_phone,t,...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90


In [201]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
ordinal=OrdinalEncoder(categories=[['f', 't']])

#Se transforman los datos 
df['host_is_superhost']=ordinal.fit_transform(df[['host_is_superhost']])

#Cambiar el tipo de dato a un entero 
df['host_is_superhost']=df['host_is_superhost'].astype('int64')
df

,last_scraped,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_verifications,host_identity_verified,...,availability_60,availability_90,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_checkin,review_scores_communication,review_scores_location
0,2024-12-20,2008-12-03,"Massachusetts, United States",within a day,100.0,22.0,1,East Boston,email_phone,f,...,0,17,26.0,1.0,0.0,4.96,5.00,5.00,4.96,4.85
1,2024-12-20,2009-02-19,"Boston, MA",within an hour,100.0,99.0,1,Roxbury,email_phone,t,...,50,80,132.0,10.0,0.0,4.81,4.88,4.95,4.89,4.56
2,2024-12-20,2009-02-19,"Boston, MA",within an hour,100.0,99.0,1,Roxbury,email_phone,t,...,52,82,135.0,8.0,0.0,4.81,4.83,4.91,4.95,4.53
3,2024-12-20,2009-07-22,"Boston, MA",within a few hours,100.0,49.0,1,Beacon Hill,email_phone,t,...,0,0,29.0,1.0,0.0,4.69,4.69,4.93,4.97,4.97
4,2024-12-20,2009-09-16,"Boston, MA",within a day,91.0,58.0,0,Back Bay,email_phone,t,...,59,89,9.0,0.0,0.0,4.80,4.90,4.90,4.90,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4421,2024-12-20,2020-10-28,"Boston, MA",within an hour,100.0,97.0,0,Harvard Square,email_phone,t,...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4422,2024-12-20,2020-10-28,"Boston, MA",within an hour,100.0,97.0,0,Harvard Square,email_phone,t,...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4423,2024-12-20,2018-08-29,"Boston, MA",within an hour,100.0,98.0,0,Theater District,email_phone,t,...,57,87,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4424,2024-12-20,2016-05-29,"Boston, MA",within an hour,100.0,100.0,1,Roslindale,email_phone,t,...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90


In [202]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
ordinal=OrdinalEncoder(categories=[['email_phone',
    'email_phone_work_email',
    'phone',
    'phone_work_email',
    'email']])

#Se transforman los datos 
df['host_verifications']=ordinal.fit_transform(df[['host_verifications']])

#Cambiar el tipo de dato a un entero 
df['host_verifications']=df['host_verifications'].astype('int64')
df

,last_scraped,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_verifications,host_identity_verified,...,availability_60,availability_90,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_checkin,review_scores_communication,review_scores_location
0,2024-12-20,2008-12-03,"Massachusetts, United States",within a day,100.0,22.0,1,East Boston,0,f,...,0,17,26.0,1.0,0.0,4.96,5.00,5.00,4.96,4.85
1,2024-12-20,2009-02-19,"Boston, MA",within an hour,100.0,99.0,1,Roxbury,0,t,...,50,80,132.0,10.0,0.0,4.81,4.88,4.95,4.89,4.56
2,2024-12-20,2009-02-19,"Boston, MA",within an hour,100.0,99.0,1,Roxbury,0,t,...,52,82,135.0,8.0,0.0,4.81,4.83,4.91,4.95,4.53
3,2024-12-20,2009-07-22,"Boston, MA",within a few hours,100.0,49.0,1,Beacon Hill,0,t,...,0,0,29.0,1.0,0.0,4.69,4.69,4.93,4.97,4.97
4,2024-12-20,2009-09-16,"Boston, MA",within a day,91.0,58.0,0,Back Bay,0,t,...,59,89,9.0,0.0,0.0,4.80,4.90,4.90,4.90,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4421,2024-12-20,2020-10-28,"Boston, MA",within an hour,100.0,97.0,0,Harvard Square,0,t,...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4422,2024-12-20,2020-10-28,"Boston, MA",within an hour,100.0,97.0,0,Harvard Square,0,t,...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4423,2024-12-20,2018-08-29,"Boston, MA",within an hour,100.0,98.0,0,Theater District,0,t,...,57,87,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90
4424,2024-12-20,2016-05-29,"Boston, MA",within an hour,100.0,100.0,1,Roslindale,0,t,...,60,90,0.0,0.0,0.0,4.80,4.90,4.90,4.90,4.90


In [203]:
#Almacenamos las columnas que elegiré para el ejercicio
colum = ['price', 'room_type', 'has_availability', 'host_acceptance_rate', 'host_is_superhost', 'host_verifications']

# O sobreescribes el mismo DataFrame
df = df[colum]
df

,price,room_type,has_availability,host_acceptance_rate,host_is_superhost,host_verifications
0,NaN,0,0,22.0,1,0
1,112.0,0,0,99.0,1,0
2,122.0,0,0,99.0,1,0
3,125.0,0,0,49.0,1,0
4,110.0,0,0,58.0,0,0
...,...,...,...,...,...,...
4421,41.0,1,0,97.0,0,0
4422,61.0,1,0,97.0,0,0
4423,132.0,0,0,98.0,0,0
4424,145.0,0,0,100.0,1,0


In [204]:
#Rellenamos valores nulos para mejores resultados
df.fillna(method='ffill', inplace=True)
df.isnull().sum()
df.loc[0, 'price'] = df['price'].mean()
df.isnull().sum()

/var/folders/7z/476dfqk54njct4l35klw1ylc0000gn/T/ipykernel_28164/176705033.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
/var/folders/7z/476dfqk54njct4l35klw1ylc0000gn/T/ipykernel_28164/176705033.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(method='ffill', inplace=True)


price                   0
room_type               0
has_availability        0
host_acceptance_rate    0
host_is_superhost       0
host_verifications      0
dtype: int64

In [205]:
CF1 = df.corr()
CF1= abs(CF1)
CF1

,price,room_type,has_availability,host_acceptance_rate,host_is_superhost,host_verifications
price,1.000000,0.124001,0.008874,0.012318,0.004008,0.033320
room_type,0.124001,1.000000,0.024378,0.103240,0.004476,0.150868
has_availability,0.008874,0.024378,1.000000,0.004195,0.020897,0.016596
host_acceptance_rate,0.012318,0.103240,0.004195,1.000000,0.104710,0.007202
host_is_superhost,0.004008,0.004476,0.020897,0.104710,1.000000,0.108886
host_verifications,0.033320,0.150868,0.016596,0.007202,0.108886,1.000000


In [206]:
CF1.to_csv('CORRECLACIONES.csv')

In [207]:
copy1 = df.copy()
copy2 = df.copy()
copy3 = df.copy()
copy4 = df.copy()
copy5 = df.copy()
copy6 = df.copy()
copy7 = df.copy()
copy8 = df.copy()
copy9 = df.copy()
copy10 = df.copy()

In [208]:
umbral = copy1['price'].mean()  
copy1['price'] = (copy1['price'] >= umbral).astype(int)


In [209]:
copy1['price'].unique()

array([0, 1])

### Por el promedio de precio y superiores, e inferiores al promedio

In [210]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=copy1[['room_type', 'host_acceptance_rate', 'has_availability']]
vars_Dep=copy1[['price']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression(class_weight='balanced')

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label =0)
precision2 = precision_score(y_test, y_pred, average='binary', pos_label =1)
print('primer precision del modelo: ')
print(precision)
print('')
print('segunda precision del modelo: ')
print(precision2)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label=0)
sensibilidad2 = recall_score(y_test, y_pred, average='binary', pos_label=1)
print('primer sensibilidad del modelo: ')
print(sensibilidad)
print('')
print('segunda sensibilidad del modelo: ')
print(sensibilidad2)

Matriz de confusión: 
[[377 559]
 [ 63 329]]

primer precision del modelo: 
0.8568181818181818

segunda precision del modelo: 
0.3704954954954955

exactitud del modelo
0.5316265060240963

primer sensibilidad del modelo: 
0.4027777777777778

segunda sensibilidad del modelo: 
0.8392857142857143


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [211]:
copy1['room_type'].unique()

array([0, 1, 3, 2])

In [212]:
copy2['room_type']= copy2['room_type'].replace([1,2,3], 4)
copy3['room_type']= copy3['room_type'].replace([0,2,3], 4)
copy4['room_type']= copy4['room_type'].replace([0,1,3], 4)
copy5['room_type']= copy5['room_type'].replace([0,1,2], 4)

### Entire home/apt

In [213]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=copy2[['host_acceptance_rate', 'price', 'host_is_superhost']]
vars_Dep=copy2[['room_type']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression(class_weight='balanced')

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label =0)
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label=0)
print('sensibilidad del modelo: ')
print(sensibilidad)


Matriz de confusión: 
[[540 356]
 [ 91 341]]

precision del modelo: 
0.8557844690966719

exactitud del modelo
0.6634036144578314

sensibilidad del modelo: 
0.6026785714285714


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Private room

In [214]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=copy3[['host_acceptance_rate', 'price', 'host_is_superhost']]
vars_Dep=copy3[['room_type']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression(class_weight='balanced')

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label =1)
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label=1)
print('sensibilidad del modelo: ')
print(sensibilidad)


Matriz de confusión: 
[[341  78]
 [320 589]]

precision del modelo: 
0.5158850226928896

exactitud del modelo
0.7003012048192772

sensibilidad del modelo: 
0.8138424821002387


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Hotel room

In [215]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=copy4[['host_acceptance_rate', 'price', 'host_is_superhost']]
vars_Dep=copy4[['room_type']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression(class_weight='balanced')

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label =2)
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label=2)
print('sensibilidad del modelo: ')
print(sensibilidad)

Matriz de confusión: 
[[  11    3]
 [ 209 1105]]

precision del modelo: 
0.05

exactitud del modelo
0.8403614457831325

sensibilidad del modelo: 
0.7857142857142857


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Shared room

In [216]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=copy5[['host_acceptance_rate', 'price', 'host_is_superhost']]
vars_Dep=copy5[['room_type']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression(class_weight='balanced')

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label =3)
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label=3)
print('sensibilidad del modelo: ')
print(sensibilidad)

Matriz de confusión: 
[[  0   1]
 [357 970]]

precision del modelo: 
0.0

exactitud del modelo
0.7304216867469879

sensibilidad del modelo: 
0.0


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Host is super host

In [217]:
copy6['host_is_superhost'].unique()

array([1, 0])

In [218]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=copy6[['host_verifications', 'price', 'has_availability']]
vars_Dep=copy6[['host_is_superhost']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression(class_weight='balanced')

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label =1)
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label=1)
print('sensibilidad del modelo: ')
print(sensibilidad)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Matriz de confusión: 
[[279 619]
 [100 330]]

precision del modelo: 
0.34773445732349845

exactitud del modelo
0.4585843373493976

sensibilidad del modelo: 
0.7674418604651163


### Has availability

In [219]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=copy7[['price', 'host_is_superhost', 'room_type']]
vars_Dep=copy7[['has_availability']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression(class_weight='balanced')

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label =0)
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label=0)
print('sensibilidad del modelo: ')
print(sensibilidad)

Matriz de confusión: 
[[757 569]
 [  0   2]]

precision del modelo: 
1.0

exactitud del modelo
0.5715361445783133

sensibilidad del modelo: 
0.5708898944193062


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Host acceptance rate por los que están sobre el promedio y debajo del mismo

In [220]:
umbral = copy8['host_acceptance_rate'].mean()  
copy8['host_acceptance_rate'] = (copy8['host_acceptance_rate'] >= umbral).astype(int)
copy8['host_acceptance_rate'].unique()

array([0, 1])

In [221]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=copy8[['host_is_superhost', 'price', 'host_verifications', 'has_availability']]
vars_Dep=copy8[['host_acceptance_rate']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression(class_weight='balanced')

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label =0)
precision2 = precision_score(y_test, y_pred, average='binary', pos_label =1)
print('primer precision del modelo: ')
print(precision)
print('')
print('segunda precision del modelo: ')
print(precision2)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label=0)
sensibilidad2 = recall_score(y_test, y_pred, average='binary', pos_label=1)
print('primer sensibilidad del modelo: ')
print(sensibilidad)
print('')
print('segunda sensibilidad del modelo: ')
print(sensibilidad2)

Matriz de confusión: 
[[134 136]
 [373 685]]

primer precision del modelo: 
0.26429980276134124

segunda precision del modelo: 
0.8343483556638246

exactitud del modelo
0.6167168674698795

primer sensibilidad del modelo: 
0.4962962962962963

segunda sensibilidad del modelo: 
0.6474480151228733


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Verificaciones que posee el host (solo lo aplicaremos para 'email_phone', ya que es la más releevante)

In [222]:
copy9['host_verifications']= copy9['host_verifications'].replace([1,2,3,4], 5)

In [223]:
#Declaramos las variables dependientes e independietes para la regresión lineal 
vars_Indep=copy9[['host_is_superhost', 'has_availability', 'host_acceptance_rate']]
vars_Dep=copy9[['host_verifications']]

#Redefinimos las variables
X= vars_Indep
y= vars_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan todos los datos : Escalr es como si pasaran los datos a una sola escala en porcentajes, como una regla de tres 
escalar = StandardScaler()

#Para realuzar el escalamiento de las variables "X" tanto en entrenamiento como de prueba utilizaremos:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression(class_weight='balanced')

#Entrenamos el modelo

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) #Predecimos los datos de test
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión: ')
print(matriz)
print('')

#Calculo la precisión de mmodelo

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='binary', pos_label =0)
print('precision del modelo: ')
print(precision)
print('')


#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('exactitud del modelo')
print(exactitud)
print('')


#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label=0)
print('sensibilidad del modelo: ')
print(sensibilidad)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Matriz de confusión: 
[[384 587]
 [ 95 262]]

precision del modelo: 
0.8016701461377871

exactitud del modelo
0.4864457831325301

sensibilidad del modelo: 
0.39546858908341914
